# Working with Pandas and XArray

This notebook demonstrates how Pandas and XArray can be used to work with the [QCoDeS DataSet](DataSet-class-walkthrough.ipynb). It is not meant as a general introduction to Pandas and XArray. We refer to the official documentation for [Pandas](https://pandas.pydata.org/) and [XArray](http://xarray.pydata.org/en/stable/) for this. This notebook requires that both Pandas and XArray are installed.

## Setup

First we borrow an example from the measurement notebook to have some data to work with. We split the measurement in two so we can try merging it with Pandas.

In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import qcodes as qc
from qcodes import load_or_create_experiment, initialise_database, Measurement
from qcodes.tests.instrument_mocks import DummyInstrument

qc.logger.start_all_logging()

Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-halakh\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-halakh\.qcodes\logs\200415-23360-qcodes.log
Activating auto-logging. Current session state plus future input saved.
Filename       : C:\Users\a-halakh\.qcodes\logs\command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active
Qcodes Logfile : C:\Users\a-halakh\.qcodes\logs\200415-23360-qcodes.log


In [2]:
# preparatory mocking of physical setup
dac = DummyInstrument('dac', gates=['ch1', 'ch2'])
dmm = DummyInstrument('dmm', gates=['v1', 'v2'])
station = qc.Station(dmm, dac)

In [3]:
initialise_database()
exp = load_or_create_experiment(experiment_name='working_with_pandas',
                          sample_name="no sample")

In [4]:
meas = Measurement(exp)
meas.register_parameter(dac.ch1)  # register the first independent parameter
meas.register_parameter(dac.ch2)  # register the second independent parameter
meas.register_parameter(dmm.v1, setpoints=(dac.ch1, dac.ch2))  # register the dependent one

In [5]:
# dmm_parameter simulate a physical signal, in this case an exponentially
# decaying signal 

class dmm_gauss_parameter(qc.Parameter):
    def __init__(self, name):
        super().__init__(name)
        self.gauss = self.gauss_model(0.1, 0.2, 0.25)
        next(self.gauss)


    def get_raw(self):
        """
        This method is automatically wrapped to
        provide a ``get`` method on the parameter instance.
        """
        val = self.gauss.send((dac.ch1.get(), dac.ch2.get()))
        next(self.gauss)
        return val

    def gauss_model(self, x0: float, y0: float, sigma: float, noise: float=0.0005):
        """
        Returns a generator sampling a gaussian. The gaussian is
        normalised such that its maximal value is simply 1
        """
        while True:
            (x, y) = yield
            model = np.exp(-((x0-x)**2+(y0-y)**2)/2/sigma**2)*np.exp(2*sigma**2)
            noise = np.random.randn()*noise
            yield model + noise

    
dmm.v1 = dmm_gauss_parameter('dmm_v1')

We then perform a very basic experiment. To be able to demonstrate merging of datasets in Pandas we will perform the measurement in two parts.

In [6]:
# run a 2D sweep

with meas.run() as datasaver:

    for v1 in np.linspace(-1, 0, 200, endpoint=False):
        for v2 in np.linspace(-1, 1, 201):
            dac.ch1(v1)
            dac.ch2(v2)
            val = dmm.v1.get()
            datasaver.add_result((dac.ch1, v1),
                                 (dac.ch2, v2),
                                 (dmm.v1, val))
            
df1 = datasaver.dataset.get_data_as_pandas_dataframe()['dmm_v1']

Starting experimental run with id: 237. 


In [7]:
# run a 2D sweep

with meas.run() as datasaver:

    for v1 in np.linspace(0, 1, 201):
        for v2 in np.linspace(-1, 1, 201):
            dac.ch1(v1)
            dac.ch2(v2)
            val = dmm.v1.get()
            datasaver.add_result((dac.ch1, v1),
                                 (dac.ch2, v2),
                                 (dmm.v1, val))
            
df2 = datasaver.dataset.get_data_as_pandas_dataframe()['dmm_v1']

2020-04-15 10:25:37,830 ¦ qcodes.dataset.sqlite.connection ¦ ERROR ¦ connection ¦ atomic ¦ 106 ¦ Rolling back due to unhandled exception
Traceback (most recent call last):
  File "c:\users\a-halakh\documents\microsoft\qcodes\qcodes\dataset\sqlite\connection.py", line 103, in atomic
    yield conn
  File "c:\users\a-halakh\documents\microsoft\qcodes\qcodes\dataset\sqlite\queries.py", line 1146, in _insert_run
    parent_dataset_links)
  File "c:\users\a-halakh\documents\microsoft\qcodes\qcodes\dataset\sqlite\connection.py", line 134, in transaction
    c.execute(sql, args)
sqlite3.OperationalError: database is locked
2020-04-15 10:25:37,840 ¦ qcodes.dataset.sqlite.connection ¦ ERROR ¦ connection ¦ atomic ¦ 106 ¦ Rolling back due to unhandled exception
Traceback (most recent call last):
  File "c:\users\a-halakh\documents\microsoft\qcodes\qcodes\dataset\sqlite\connection.py", line 103, in atomic
    yield conn
  File "c:\users\a-halakh\documents\microsoft\qcodes\qcodes\dataset\sqlite\que

RuntimeError: Rolling back due to unhandled exception

`get_data_as_pandas_dataframe` returns the data as a dict from measured (dependent) parameters to DataFrames. Here we are only interested in the dataframe of a single parameter, so we select that from the dict.

## Working with Pandas

Lets first inspect the Pandas DataFrame. Note how both dependent variables are used for the index. Pandas refers to this as a [MultiIndex](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html). For visual clarity, we just look at the first N points of the dataset.

In [ ]:
N = 10

In [ ]:
df1[:N]

We can also reset the index to return a simpler view where all data points are simply indexed by a running counter. As we shall see below this can be needed in some situations. Note that calling `reset_index` leaves the original dataframe untouched.

In [ ]:
df1.reset_index()[0:N]

Pandas has built-in support for various forms of plotting. This does not, however, support MultiIndex at the moment so we use `reset_index` to make the data available for plotting.

In [ ]:
df1.reset_index().plot.scatter('dac_ch1', 'dac_ch2', c='dmm_v1')

Similarly, for the other dataframe:

In [ ]:
df2.reset_index().plot.scatter('dac_ch1', 'dac_ch2', c='dmm_v1')

Merging two dataframes with the same labels is fairly simple.

In [ ]:
df = pd.concat([df1, df2], sort=True)

In [ ]:
df.reset_index().plot.scatter('dac_ch1', 'dac_ch2', c='dmm_v1')

It is also possible to select a subset of data from the datframe based on the x and y values.

In [ ]:
df.loc[(slice(-1, -0.95), slice(-1, -0.97)), :]

## Working with XArray

In many cases when working with data on a rectangular grids it may be more convenient to export the data to a [XArray](http://xarray.pydata.org) Dataset or DataArray

The Pandas DataSet can be directly converted to a XArray [Dataset](http://xarray.pydata.org/en/stable/data-structures.html?#dataset):

In [ ]:
xaDataSet = df.to_xarray()

In [ ]:
xaDataSet

However, in many cases it is more convenient to work with a XArray [DataArray](http://xarray.pydata.org/en/stable/data-structures.html?#dataarray). The DataArray can only contain a single dependent variable and can be obtained from the Dataset by indexing using the parameter name.

In [ ]:
xaDataArray = xaDataSet['dmm_v1']

In [ ]:
xaDataArray

In [ ]:
fig, ax = plt.subplots(2,2)
xaDataArray.plot(ax=ax[0,0])
xaDataArray.mean(dim='dac_ch1').plot(ax=ax[1,0])
xaDataArray.mean(dim='dac_ch2').plot(ax=ax[0,1])
xaDataArray[200,:].plot(ax=ax[1,1])
fig.tight_layout()

Above we demonstrated a few ways to index the data from a DataArray. For instance the DataArray can be directly plotted, the extracted mean or a specific row/column can also be plotted.